In [4]:
import numpy as np
from qiskit.circuit import ParameterVector
from qiskit import QuantumCircuit
from qiskit.circuit import Gate

class SqueezingGate(Gate):
    """
    Phase space squeezing gate for continuous variables.

    S(z) = exp(1/2 * (z^* a^2 - z a^†^2)),

    where z = r * exp(i * phi).
    """

    def __init__(self, r, phi):
        # Initialize as a gate with 2 parameters (r and phi) acting on 1 qubit
        super().__init__("S", 1, [r, phi])

    def _define(self):
        """
        Define the decomposition of this gate in terms of more fundamental gates.
        """
        r, phi = self.params

        # Creating the corresponding Heisenberg representation matrix for the squeezing gate
        cosh_r = np.cosh(r)
        sinh_r = np.sinh(r)
        M = np.array([
            [1, 0, 0],
            [0, cosh_r - np.cos(phi) * sinh_r, -np.sin(phi) * sinh_r],
            [0, -np.sin(phi) * sinh_r, cosh_r + np.cos(phi) * sinh_r]
        ])

        # Add this unitary as a custom gate to the quantum circuit
        q = QuantumCircuit(1)
        q.unitary(M, 0)

        self.definition = q

    def inverse(self):
        """Returns the adjoint (inverse) of the squeezing gate."""
        r, phi = self.params
        new_phi = (phi + np.pi) % (2 * np.pi)
        return SqueezingGate(r, new_phi)
    
class SqueezingEmbedding:
    def __init__(self, num_qubits: int, amplitude: bool = True):
        """
        Class for embedding a data vector into a quantum circuit using SqueezingGates.

        Args:
            num_qubits (int): The number of qubits in the circuit.
            amplitude (bool): If True, the squeezing amplitude will be parameterized and the phase will be fixed at 0.1.
                              If False, the phase will be parameterized and the squeezing amplitude will be fixed at 0.1.
        """
        self.num_qubits = num_qubits
        self.amplitude = amplitude
        self.parameters = ParameterVector('x', length=num_qubits)

    def create_circuit(self):
        """
        Creates and returns a quantum circuit with the specified SqueezingGates applied to each qubit.

        Returns:
            QuantumCircuit: The generated quantum circuit.
        """
        qc = QuantumCircuit(self.num_qubits)

        for i in range(self.num_qubits):
            if self.amplitude:
                # Parameterize the squeezing amplitude with fixed phase
                r = self.parameters[i]
                phi = 0.1
            else:
                # Parameterize the squeezing phase with fixed amplitude
                r = 0.1
                phi = self.parameters[i]

            squeezing_gate = SqueezingGate(r, phi)
            qc.append(squeezing_gate, [i])

        return qc

In [7]:
embedding = SqueezingEmbedding(num_qubits=3, amplitude=True)
circuit = embedding.create_circuit()
circuit.draw()

┌─────────────┐
q_0: ┤ S(x[0],0.1) ├
     ├─────────────┤
q_1: ┤ S(x[1],0.1) ├
     ├─────────────┤
q_2: ┤ S(x[2],0.1) ├
     └─────────────┘